In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
import timm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from transformers import AutoProcessor, CLIPModel
import torchvision.transforms as T
from tqdm import tqdm

DATA_DIR = '../SnakeCLEF2023-medium_size-train/'
VAL_DIR='../SnakeCLEF2023-medium_size-val'
SEED = 42


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cpu


In [15]:
# load metadata
train_df = pd.read_csv('../snake_csv_files/SnakeCLEF_bbox_annotations_train_val_rare_diet.csv')
valid_df = pd.read_csv('../snake_csv_files/SnakeCLEF2023-cleaned-metadata-val.csv')


train_df.head()
classes = np.unique(train_df['binomial'])
no_classes = len(classes)
print(f'No classes: {no_classes}')
print(f'Train set length: {len(train_df):,d}')
print(f'Validation set length: {len(valid_df):,d}')

No classes: 1784
Train set length: 145,381
Validation set length: 10,985


/var/folders/k1/my0k39b95sg30r6_k0vgqk9c0000gn/T/ipykernel_30528/336587869.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('../snake_csv_files/SnakeCLEF_bbox_annotations_train_val_rare_diet.csv')


In [16]:
train_df.head()



,image_path,observation_id,binomial,code,endemic,class_id,xmin,ymin,xmax,ymax
0,1990/Zamenis_lineatus/3001242.jpg,2670823,Zamenis lineatus,IT,True,1779,81,7,377,319
1,1990/Xenoxybelis_argenteus/113910655.jpg,70108926,Xenoxybelis argenteus,VE,False,1772,52,6,350,383
2,1990/Xenoxybelis_argenteus/113910659.jpg,70108926,Xenoxybelis argenteus,VE,False,1772,0,146,379,280
3,1990/Aspidelaps_lubricus/168477.JPG,117935,Aspidelaps lubricus,ZA,False,95,4,68,364,316
4,1990/Telescopus_beetzi/177365.JPG,125284,Telescopus beetzi,ZA,False,1606,23,1,364,363


In [17]:
train_df.columns

Index(['image_path', 'observation_id', 'binomial', 'code', 'endemic',
       'class_id', 'xmin', 'ymin', 'xmax', 'ymax'],
      dtype='object')

In [18]:
'''
CLIP MODEL 

'''


model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [19]:
import csv
fh = open('clip_embeddings.csv', 'w')
cvs_writer = csv.writer(fh)

# write one row with headers (using `writerow` without `s` at the end)
cvs_writer.writerow(['CLIP_embedding', 'image_path', 'observation_id', 'binomial', 'code', 'endemic',
       'class_id', 'xmin', 'ymin', 'xmax', 'ymax'])
                                                                                                
fh.close()

In [ ]:

for index, row in train_df.iterrows():
    # print(index)
    image = Image.open(DATA_DIR+row['image_path'])
    inputs = processor(images=image, return_tensors="pt")
    image_features = model.get_image_features(**inputs)

    result = [image_features.tolist(), row['image_path'], row['observation_id'], row['binomial'], row['code'], row['endemic'],
       row['class_id'], row['xmin'], row['ymin'], row['xmax'], row['ymax']]

    fh = open('clip_embeddings.csv', 'a') 
    cvs_writer = csv.writer(fh)

    # write row row with result (using `writerow` without `s` at the end)
    cvs_writer.writerow(result)
    print(' Done {}'.format(row["image_path"]))

    fh.close()




In [3]:
patch_h = 40
patch_w = 40
feat_dim = 1536 # vitg14

'''
DINOV2 MODEL
'''
dinov2_vitg14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14').to(device)
 


Using cache found in /Users/aartibalana/.cache/torch/hub/facebookresearch_dinov2_main
xFormers not available
xFormers not available


In [4]:


'''
Dataset class 

'''
class SnakeTrainDataset(Dataset):

    def __init__(self, data, transform = None):
        self.data=data
        self.transform=transform

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):

        image = self.data.iloc[index]
        img = Image.open(DATA_DIR+image.image_path).convert("RGB")
        # img=TF.adjust_sharpness(img, 20.0)
        label = torch.tensor(image.class_id)

        if self.transform is not None:
            img = self.transform(img)

        return (img, label)

In [5]:
transform_train = T.Compose([
    T.GaussianBlur(9, sigma=(0.1, 2.0)),
    T.Resize((patch_h * 14, patch_w * 14)),
    T.CenterCrop((patch_h * 14, patch_w * 14)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

train_dataset = SnakeTrainDataset(train_df, transform=transform_train) # data augmentation. set augmentations = None to disable augmentations

train_dataloader = DataLoader(train_dataset, batch_size = 6, shuffle = False)


In [ ]:
for i, (imgs, targs) in enumerate(tqdm(train_dataloader)):
        imgs = imgs.to(device)
        targs = targs.to(device)
        with torch.no_grad():
            features=dinov2_vitg14.forward_features(imgs)['x_norm_clstoken']

            print(features.shape)

In [1]:
'''
CONVNEXT XXLARGE

'''

model = timm.create_model(
    'convnext_xxlarge.clip_laion2b_soup_ft_in1k',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)



In [ ]:
for index, row in train_df.iterrows():

    image = Image.open(DATA_DIR+row['image_path'])

    output = model(transforms(image).unsqueeze(0))  


    output = model.forward_features(transforms(image).unsqueeze(0))
    # output is unpooled, a (1, 3072, 8, 8) shaped tensor

    output = model.forward_head(output, pre_logits=True)
    # output is a (1, num_features) shaped tensor

    print(output.shape)
    print(row['binomial'])

In [ ]:
'''
MAke a dataset function and dataloader which outputs concatenated embeddings and class_id and binomial_names
'''

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Example image embeddings of shape 
'''
TODO :
'''
embedding_1 = 3#CLIP
embedding_2 = 3 #DINO
embedding_3 = 3#CONVXNEXT

# Concatenate the embeddings
concatenated_embeddings = torch.cat([embedding_1, embedding_2, embedding_3], dim=0)

# Define the classifier
num_classes = 1784  # Example: number of classes
classifier = nn.Sequential(
    nn.Linear(728 + 512 + 256 + 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, num_classes)
)


# Set up optimizer and loss function
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0

    for images, labels in dataloader: #add dataloader
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        embeddings = torch.cat([embedding_1, embedding_2, embedding_3], dim=1)
        logits = classifier(embeddings)

        # Calculate loss
        loss = criterion(logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
